In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import chromedriver_binary
import string

pd.options.display.float_format = '{:.0f}'.format

In [2]:
is_link = 'https://finance.yahoo.com/quote/AAPL/financials?p=AAPL'
bs_link = 'https://finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL'
cf_link = 'https://finance.yahoo.com/quote/AAPL/cash-flow?p=AAPL'

In [3]:
# ticker = input('Enter your company ticker symbol here: ')
# link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'

In [4]:
driver = webdriver.Chrome()
driver.get(is_link)
html = driver.execute_script('return document.body.innerHTML;')
soup = BeautifulSoup(html,'lxml')

In [5]:
close_price = [entry.text for entry in soup.find_all('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]
after_hours_price = [entry.text for entry in soup.find_all('span', {'class':'C($primaryColor) Fz(24px) Fw(b)'})]

In [6]:
close_price

['309.51']

In [7]:
features = soup.find_all('div', class_='D(tbr)')

In [8]:
headers = []
temp_list = []
label_list = []
final = []
index = 0

#create headers
for item in features[0].find_all('div', class_='D(ib)'):
    headers.append(item.text)

#statement contents
while index <= len(features)-1:
    #filter for each line of the statement
    temp = features[index].find_all('div', class_='D(tbc)')
    for line in temp:
        #each item adding to a temporary list
        temp_list.append(line.text)
    #temp_list added to final list
    final.append(temp_list)
    #clear temp_list
    temp_list = []
    index+=1

In [9]:
df = pd.DataFrame(final[1:])
df.columns = headers

In [10]:
#function to make all values numerical (or - for NaNs)
def convert_to_numeric(column):

    first_col = [i.replace(',','') for i in column]
    second_col = [i.replace('-','') for i in first_col]
    final_col = pd.to_numeric(second_col)
    
    return final_col

In [11]:
for column in headers[1:]:
    
    df[column] = convert_to_numeric(df[column])

In [12]:
final_df = df.fillna('-')

In [13]:
final_df

,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017,9/30/2016
0,Total Revenue,267683000,260174000,265595000,229234000,215639000
1,Cost of Revenue,166105000,161782000,163756000,141048000,131376000
2,Gross Profit,101578000,98392000,101839000,88186000,84263000
3,Operating Expenses,-,-,-,-,-
4,Research Development,16766000,16217000,14236000,11581000,10045000
5,Selling General and Administrative,18659000,18245000,16705000,15261000,14194000
6,Total Operating Expenses,35425000,34462000,30941000,26842000,24239000
7,Operating Income or Loss,66153000,63930000,70898000,61344000,60024000
8,Interest Expense,-,3576000,3240000,2323000,1456000
9,Total Other Income/Expenses Net,628000,1807000,441000,133000,1195000


In [15]:
# ratios
# value a company
# investment analysis - quantitative financial analysis, look at trends in all of their metrics
# sales growth obvious, but like for Amazon, their cash flow from payables/receivables
# what's the intrinsic value of apple and other companies? make it generalizable so I can apply this 
# make a formula to make a projection baseline for companies and their Free Cash Flow and discount it back